In [9]:
!pip install gradio
import gradio as gr  # Import Gradio for creating the web interface

In [10]:
# #@title Example: Reading and Displaying an Image
# from google.colab import files
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# uploaded = files.upload()  # This will prompt you to upload an image file

# # Let's assume the uploaded file is named test_image.jpg
# img_path = list(uploaded.keys())[0]

# img=cv2.imread(img_path)
# img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

# plt.figure(figsize=(8,6))
# plt.imshow(img_rgb)
# plt.title('Original Image')
# plt.axis('off')
# plt.show()

IndexError: list index out of range

In [11]:
import json
import requests
import base64
from google.colab import files
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io
# Define API key for authentication
apiKey = ''
# Base URL for the API endpoint
basicUrl = "https://genai.hkbu.edu.hk/general/rest"
# Specify the model name to be used
modelName = "gpt-4-o"
# API version to be used in the request
apiVersion = "2024-10-21"

In [12]:
# Define function to submit vision prompt with an uploaded image
def submit_vision_prompt(image_path, prompt_text):
    # Convert image to base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    # Create the message payload with both text and image
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_text},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}", "detail": "high"}}
            ]
        }
    ]

    # Construct the full API URL
    url = basicUrl + "/deployments/" + modelName + "/chat/completions/?api-version=" + apiVersion

    # Set headers with content type and API key
    headers = {'Content-Type': 'application/json', 'api-key': apiKey}

    # Create payload with the conversation messages
    payload = {'messages': conversation}

    # Send POST request to the API
    response = requests.post(url, json=payload, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        # Extract the assistant's reply
        assistant_reply = data["choices"][0]["message"]["content"]
        return assistant_reply
    else:
        # Return error message if request fails
        return f"Error: {response.status_code} - {response.text}"

In [13]:
# Function to analyze food image and estimate calories
def predict(image):
    # Convert the image to a format we can use
    if isinstance(image, np.ndarray):
        pil_image = Image.fromarray(image)
    else:
        pil_image = image

    # Convert PIL image to base64 string
    buffer = io.BytesIO()
    pil_image.save(buffer, format="JPEG")
    base64_image = base64.b64encode(buffer.getvalue()).decode('utf-8')

    # Create the prompt for food identification
    food_prompt = """Identify all food items in this image and provide:
    1) List of all foods
    2) Estimated calories per serving for each item
    3) Approximate portion sizes
    4) Total calories for the meal"""

    # Create the message payload with both text and image
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": food_prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}", "detail": "high"}}
            ]
        }
    ]

    # Construct the full API URL
    url = basicUrl + "/deployments/" + modelName + "/chat/completions/?api-version=" + apiVersion

    # Set headers with content type and API key
    headers = {'Content-Type': 'application/json', 'api-key': apiKey}

    # Create payload with the conversation messages
    payload = {'messages': conversation}

    # Send POST request to the API
    try:
        response = requests.post(url, json=payload, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            # Extract the assistant's reply
            assistant_reply = data["choices"][0]["message"]["content"]
            return format_results(assistant_reply)
        else:
            # Return error message if request fails
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error processing image: {str(e)}"

In [14]:
# Format the results for better readability
def format_results(analysis_text):
    formatted_result = "🍽️ FOOD ANALYSIS RESULTS 🍽️\n"
    formatted_result += "=" * 40 + "\n\n"
    formatted_result += analysis_text
    return formatted_result


In [15]:
# Create the Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🥗 Food Calorie Estimator")
    gr.Markdown("Upload a food image to get calorie estimates and nutritional information")

    with gr.Row():
        with gr.Column():
            input_image = gr.Image(type="pil", label="Upload Food Image")
            submit_btn = gr.Button("Analyze Food", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Analysis Results", lines=15)


    # Set up the button click event
    submit_btn.click(fn=predict, inputs=input_image, outputs=output_text)

    gr.Markdown("### How It Works")
    gr.Markdown("1. Upload a clear image of your food")
    gr.Markdown("2. Click 'Analyze Food' to process the image")
    gr.Markdown("3. Get detailed calorie and portion information")

# Launch the app
demo.launch(debug=True)  # share=True creates a public link

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c187483a002bbf6599.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c187483a002bbf6599.gradio.live


In [ ]:

# # Call the function with the uploaded image and food recognition prompt
# food_prompt = "Identify all food items in this image and provide: 1) List of all foods 2) Estimated calories per serving for each item 3) Approximate portion sizes"

# interface = gr.Interface(fn=predict,
#     inputs=gr.Image(type="pil"),
#     outputs="text",
#     title="Food Calorie Estimator",
#     description="Upload a food image")

# interface.launch()
# # Get the analysis results
# analysis_results = submit_vision_prompt(img_path, food_prompt)

# # Display the results
# print("FOOD ANALYSIS RESULTS:")
# print("----------------------")
# print(analysis_results)


FOOD ANALYSIS RESULTS:
----------------------
1) List of all foods: 
   - Banana

2) Estimated calories per serving:
   - A medium banana typically contains about 105 calories.

3) Approximate portion size:
   - The portion size of the banana shown in the image is approximately one medium banana, which is about 7 to 8 inches long.


In [ ]:
# # Define a function to submit a message to the API
# def submit(image_url):
#     # Create a conversation list with a single user message
#     conversation = [{"role": "user", "content":[ {"type": "text", "text":"Identify all food items in this image and estimate their calorie content."},{"type":"image_url","image_url":{"url":message,"detail":"low"}}]}]
#     # Construct the full API URL with model name and version
#     url = basicUrl + "/deployments/" + modelName + "/chat/completions/?api-version=" + apiVersion
#     # Set headers with content type and API key
#     headers = { 'Content-Type': 'application/json', 'api-key': apiKey }
#     # Create payload with the conversation messages
#     payload = { 'messages': conversation }
#     # Send POST request to the API with payload and headers
#     response = requests.post(url, json=payload, headers=headers)

#     # Check if the request was successful (status code 200)
#     if response.status_code == 200:
#         # Parse the JSON response into a Python dictionary
#         data = response.json()
#         # Extract the assistant's reply from the response
#         assistant_reply = data["choices"][0]["message"]["content"]
#         return assistant_reply  # Return the assistant's reply
#     else:
#         # Return error message with response details if request fails
#         return f"Error: {response.status_code} - {response.text}"